In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import os
import gzip
import shutil
import struct
import urllib.request

In [102]:
import time

In [3]:
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

In [4]:
    #Download and unzip the dataset mnist if it's not already downloaded 
    #Download from http://yann.lecun.com/exdb/mnist
    mnist_folder = "C:\\Users\\USX28939\\PYTHON_CODE_BASE\\GitHub_Doc\\stanford-tensorflow-tutorials\\examples\\data\\MNST_DATA"

In [32]:
def download_one_file(download_url, 
                    local_dest, 
                    expected_byte=None, 
                    unzip_and_remove=False):
    """ 
    Download the file from download_url into local_dest
    if the file doesn't already exists.
    If expected_byte is provided, check if 
    the downloaded file has the same number of bytes.
    If unzip_and_remove is True, unzip the file and remove the zip file
    """
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' %local_dest)
    else:
        print('Downloading %s' %download_url)
        local_file, headers = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' %local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3],'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')

In [35]:
import urllib
print (urllib.request.urlopen("http://yann.lecun.com/exdb/mnist\"))


SyntaxError: EOL while scanning string literal (<ipython-input-35-d986ab718bac>, line 2)

In [36]:
def download_mnist(path):
    """ 
    Download and unzip the dataset mnist if it's not already downloaded 
    Download from http://yann.lecun.com/exdb/mnist
    """
    os.mkdir(path)
    url = 'http://yann.lecun.com/exdb/mnist/'
    filenames = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
    expected_bytes = [9912422, 28881, 1648877, 4542]

    for filename, byte in zip(filenames, expected_bytes):
        download_url = os.path.join(url, filename)
        local_dest = os.path.join(path, filename)
        print(download_url,"    -> ", local_dest)
        download_one_file(download_url, local_dest, byte, True)

In [37]:
download_mnist(mnist_folder)

http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-images-idx3-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-images-idx3-ubyte.gz
http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-labels-idx1-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz     ->  C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\MNST_DATA\t10k-images-idx3-ubyte.gz
Successfully downloaded C:\Users\USX28939\PYTHON_CODE_BASE\GitHub_Doc\stanford-tensorflow-tutorials\examples\data\M

In [5]:
def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k':
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte')
    #print(label_file )
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels

In [6]:
def read_mnist(path, flatten=True, num_train=55000):
    """
    Read in the mnist dataset, given that the data is stored in path
    Return two tuples of numpy arrays
    ((train_imgs, train_labels), (test_imgs, test_labels))
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0])
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test


In [106]:
tf.reset_default_graph() 

In [107]:
train, val, test = read_mnist(mnist_folder, flatten=True)

In [108]:
print(len(train))
print(len(val))
print(len(test))

2
2
2


In [109]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

In [110]:
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [111]:
train[0].shape

(55000, 784)

In [112]:
print(train_data.output_types)
print(train_data.output_shapes)

(tf.float32, tf.float64)
(TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [113]:
iterator = tf.data.Iterator.from_structure(train_data.output_types,train_data.output_shapes)

In [114]:
img, label = iterator.get_next()

In [115]:
print(img.shape)

(?, 784)


In [116]:
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [117]:
w = tf.get_variable(name = "Weights", dtype = tf.float32, shape = (784,10), initializer = tf.random_normal_initializer(mean=0.0,
    stddev=1.0,seed=None,dtype=tf.float32))
b = tf.get_variable(name = "Bias", dtype = tf.float32, shape = (1,1) , initializer = tf.zeros_initializer())

In [118]:
logits = tf.matmul(img,w) + b

In [119]:
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=logits)
loss = tf.reduce_mean(entropy)

In [120]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

In [121]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('C:\\Users\\USX28939\\PYTHON_CODE_BASE\\graphs', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 1.4638559681731602
Average loss epoch 1: 0.5393966492525367
Average loss epoch 2: 0.43792431721853653
Average loss epoch 3: 0.3947549240706965
Average loss epoch 4: 0.3643354202252488
Average loss epoch 5: 0.34283112041478936
Average loss epoch 6: 0.3290896705416746
Average loss epoch 7: 0.3174571279176446
Average loss epoch 8: 0.307425976700561
Average loss epoch 9: 0.3010503748475119
Average loss epoch 10: 0.30004803954168807
Average loss epoch 11: 0.2902943950752879
Average loss epoch 12: 0.2857754676321218
Average loss epoch 13: 0.28484983830604443
Average loss epoch 14: 0.2813559701796188
Average loss epoch 15: 0.2770993285747462
Average loss epoch 16: 0.2771000526152378
Average loss epoch 17: 0.2736578980850619
Average loss epoch 18: 0.27322225773403813
Average loss epoch 19: 0.27297232906485713
Average loss epoch 20: 0.2696002567057
Average loss epoch 21: 0.2698467190529025
Average loss epoch 22: 0.27074913708276527
Average loss epoch 23: 0.2670948202873385